# VGG
https://arxiv.org/pdf/1409.1556.pdf

https://www.youtube.com/watch?v=ACmuBbuXn20


### IMPORTS

In [ ]:
import matplotlib.pyplot as plt
import os
import torch
from tqdm import tqdm
from torch import nn
from torchsummary import summary
import torchvision.transforms as T
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader

### DATALOADER

In [ ]:
DATA_DIR = '/home/diogo/Documentos/IC/CNN'
CATEGORIES = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [ ]:
prep_transform = T.Compose([
                    T.ToTensor(),
                    T.Normalize(
                        (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
                    )
                  ])

tensor_train = CIFAR10(DATA_DIR, train=True, download=False,
                         transform=prep_transform)
tensor_test = CIFAR10(DATA_DIR, train=False, download=False,
                         transform=prep_transform)


In [ ]:
batch_size = 64

train_loader = DataLoader(tensor_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(tensor_test, batch_size=batch_size, shuffle=False)

### MODEL

In [ ]:
VGG16 = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

In [ ]:
class VGG_net(nn.Module):
  def __init__(self,in_channels = 3,num_classes = 10):
    
      super(VGG_net,self).__init__()
      self.in_channels = in_channels
      self.conv_layers = self.create_conv_layers(VGG16)
      self.lin_layer = nn.Sequential(
        
        nn.Linear(512,4096),
        nn.ReLU(),
        nn.Dropout(p = 0.5),
        
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Dropout(p = 0.5),
        
        nn.Linear(4096,num_classes),
      )

  def forward(self, x):
    x = self.conv_layers(x)
    x = x.reshape(x.shape[0],-1)
    x = self.lin_layer(x)
    return x
  
  def create_conv_layers(self, architecture):
    layers = []
    
    in_channels = self.in_channels
    
    for x in architecture:
      print(x)
      if type(x) == int:
        
        out_channels = x
        
        layers += [nn.Conv2d(in_channels,out_channels, kernel_size= 3, stride = 1, padding = 1),
                  nn.BatchNorm2d(x),
                  nn.ReLU()]

        in_channels = x
        
      if x == 'M':
        layers += [nn.MaxPool2d(kernel_size= 2, stride = 2)]
        
        
    return nn.Sequential(*layers)



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Rodando na {device}")

In [ ]:
cnn_model = VGG_net().to(device)

In [ ]:
summary(cnn_model, (3, 32, 32))

In [ ]:
cnn_loss_func = nn.CrossEntropyLoss()
cnn_optimizer = torch.optim.SGD(cnn_model.parameters(), lr=0.001)

### TRAINING

In [ ]:
def plot_loss(train_losses, test_losses):
    fig = plt.figure(figsize=(13,5))
    ax = fig.gca()
    ax.plot(train_losses, label="Train loss", color = "green")
    ax.plot(test_losses, label="Test loss", color = "purple")
    ax.legend(fontsize="16")
    ax.set_xlabel("Iteration", fontsize="16")
    ax.set_ylabel("Loss", fontsize="16")
    ax.set_title("Loss vs iterations", fontsize="16");
    plt.show()

In [ ]:

epochs = 51
conv_train_losses = []
conv_test_losses = []

for t in range(epochs):
    
    train_loss = 0.0
    test_loss = 0.0

    for img, label in tqdm(train_loader, desc=f'Epoch {t}/{epochs-1}'):
        
        cnn_optimizer.zero_grad()
        
        img,label = img.to(device) ,label.to(device)
        
        pred = cnn_model(img)
        loss = cnn_loss_func(pred, label)
        loss.backward()
        cnn_optimizer.step()
        
        train_loss += loss.item()
        
    train_loss = train_loss/len(train_loader)
    conv_train_losses.append(train_loss)
    
    with torch.no_grad():
        for img, labels in test_loader:
            
            img, labels = img.to(device), labels.to(device)

            pred = cnn_model(img)
            loss = cnn_loss_func(pred, labels)
            test_loss += loss.item()
        
    test_loss = test_loss / len(test_loader)
    conv_test_losses.append(test_loss)
    
    if t % 10 == 0:
        print(f"Epoch: {t}; Train Loss: {train_loss}")
        if t != 0:
            plot_loss(conv_train_losses,conv_test_losses)